# Перенос файла с диска

In [1]:
!cp "/content/drive/MyDrive/GlebHac/hackathon_part_1.mp4" ./

In [2]:
!cp "/content/drive/MyDrive/GlebHac/vox1_meta.csv" ./

In [3]:
!mkdir splited_sound

mkdir: cannot create directory ‘splited_sound’: File exists


In [4]:
!pip install pydub

In [5]:
!pip install SpeechRecognition

In [6]:
!pip install Levenshtein

In [7]:
!pip install transliterate

# Получение аудио из видео

In [8]:
from moviepy.editor import * #подключаем пакет moviepy

In [9]:
audioclip = AudioFileClip("hackathon_part_1.mp4") #видеофайл 1.mp4
audioclip.write_audiofile("out_audio.mp3") #извлеченная аудиодорожка в файл out_audio.mp3

[MoviePy] Writing audio in out_audio.mp3


100%|██████████| 4387/4387 [00:05<00:00, 784.80it/s]

[MoviePy] Done.


# Разделение аудиофайла на кусочки по n сек

In [10]:
AUDIO_FILE = "out_audio.mp3" # имя исходного файла
SEGMENT_LEN = 10 * 1000 # длинна одного отрезка 

In [11]:
from pydub import AudioSegment

In [12]:
sound = AudioSegment.from_file(AUDIO_FILE) # подгружаем файл

In [13]:
for milsec in range(0,len(sound),SEGMENT_LEN):
  segment=sound[milsec:milsec+SEGMENT_LEN] # отделяем сегмент
  segment.export("./splited_sound/s_"+str(milsec)+".wav", format="wav") # закидываем в файл

# Распознование речи


In [14]:
import speech_recognition as sr 
import IPython
from os import listdir

In [15]:
lines=[]

In [16]:
r = sr.Recognizer()

In [17]:
FILE_NAMES=sorted(os.listdir("./splited_sound"))

In [18]:
for name in FILE_NAMES:
  with sr.AudioFile("./splited_sound/"+name) as source:
    audio = r.record(source)
  lines += [r.recognize_google(audio,language="ru")]

# Поиск контрольных слов

In [19]:
import pandas as pd
controls_names = [name.split("_")[-1].lower() for name in list(pd.read_csv("vox1_meta.csv",sep="\t")["VGGFace1 ID"])]

In [20]:
import Levenshtein
# Функция, вычисляющая схожесть строк по Левенштейну
def text_sim(s1, s2) -> float:
    ratio = Levenshtein.distance(s1, s2)
    return 1.0 - ratio / max(len(s1), len(s2))


# Функция проверяющая, что слово схоже в заданной степени с хотя бы одним из слов
def is_search_word(word, true_words, min_sim=0.6) -> bool:
    word = word.lower()
    is_true_word = False
    for example_word in true_words:
        is_true_word = is_true_word or (text_sim(word, example_word) > min_sim)
    return is_true_word

In [48]:
translit_lines = [0]*len(lines)

In [58]:
from transliterate import translit
for i,line in enumerate(lines):
  translit_lines[i] =  translit(line,  language_code='ru',reversed=True).lower().split()

In [59]:
translit_lines

[['vsem',
  'privet',
  'menja',
  'zovut',
  'serezha',
  'ja',
  'rabotaju',
  'zvuki',
  'na',
  'pozitsii',
  'bacardi',
  'da',
  'ona',
  'sobaka',
  'idet',
  '12',
  'chelovek',
  'i',
  'kak',
  'i',
  'ljuboj',
  "bukan'ery",
  'kogda',
  'rabotaju',
  'ja',
  'ljublju',
  "slushat'",
  'muzyku'],
 ['chasche',
  'vsego',
  'eto',
  'dubstep',
  'ili',
  'future',
  'garage',
  'ljublju',
  "slushat'",
  'borja',
  'privet',
  'ja',
  'dima',
  'ja',
  'tehlit',
  'v',
  'produktovoj',
  'komande',
  'kotoraja',
  'zanimaetsja',
  'volnami',
  'v',
  'sverhzvuke'],
 ['klassno',
  "zamechatel'naja",
  'i',
  'kontejner',
  'tam',
  'voobsche',
  'otlichno',
  'sebja',
  'projavil',
  'no',
  "bol'she",
  'vsego',
  'mne',
  'nravitsja',
  'kamera',
  'aleksandra',
  'pistoletova',
  'ja'],
 ['na',
  'meroprijatii',
  'kotoroe',
  'predstavljaet',
  'sebja',
  'proigryvanie',
  'v',
  'simfonicheskom',
  'orkestre',
  'vo',
  'vsjakih',
  'etih',
  'saundtrekov',
  'iz',
  "fil'

In [60]:
dict(zip(FILE_NAMES,translit_lines))

{'s_0.wav': ['vsem',
  'privet',
  'menja',
  'zovut',
  'serezha',
  'ja',
  'rabotaju',
  'zvuki',
  'na',
  'pozitsii',
  'bacardi',
  'da',
  'ona',
  'sobaka',
  'idet',
  '12',
  'chelovek',
  'i',
  'kak',
  'i',
  'ljuboj',
  "bukan'ery",
  'kogda',
  'rabotaju',
  'ja',
  'ljublju',
  "slushat'",
  'muzyku'],
 's_10000.wav': ['chasche',
  'vsego',
  'eto',
  'dubstep',
  'ili',
  'future',
  'garage',
  'ljublju',
  "slushat'",
  'borja',
  'privet',
  'ja',
  'dima',
  'ja',
  'tehlit',
  'v',
  'produktovoj',
  'komande',
  'kotoraja',
  'zanimaetsja',
  'volnami',
  'v',
  'sverhzvuke'],
 's_100000.wav': ['klassno',
  "zamechatel'naja",
  'i',
  'kontejner',
  'tam',
  'voobsche',
  'otlichno',
  'sebja',
  'projavil',
  'no',
  "bol'she",
  'vsego',
  'mne',
  'nravitsja',
  'kamera',
  'aleksandra',
  'pistoletova',
  'ja'],
 's_110000.wav': ['na',
  'meroprijatii',
  'kotoroe',
  'predstavljaet',
  'sebja',
  'proigryvanie',
  'v',
  'simfonicheskom',
  'orkestre',
  'vo

In [52]:
text_sim("Timberland","Timberlake")

0.8